### 2023년 1월 1일 MMCV가 v2.0.0 출시하면서 학습 과정과 관련된 요소들을 제거하고 data 변형과 관련된 모듈을 추가할 예정 게다가 패키지 이름을 변경할 예정 mmcv-> mmcv-lite, mmcv-full -> mmcv
[Compability Document](https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md)

In [ ]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

### faster_rcnn_r50_fpn_1x_coco.py
- 구성  
 **base**  
  - ../_base_/models/faster_rcnn_r50_fpn.py  => model  
  - ../_base_/datasets/coco_detection.py  => datasets
  - ../_base_/schedules/schedule_1x.py    => schedules
  - ../_base_/default_runtime.py => 
  
#### pipeline 구성
```python
XXXX_pipeline = [  
    dict = (type , ),  
    dict = (type , ),  
]
```
위와 같은 구조를 가지고 있기 때문에 indexing 가능


#### build_XXX 위치
[github builder.py](https://github.com/open-mmlab/mmdetection/blob/e71b499608e9c3ccd4211e7c815fa20eeedf18a2/mmdet/models/builder.py#L48)


#### 코드 이해하기 위한 설명
- isinstance(object, type) -> 해당 오브젝트가 명시된 타입과 맞는 지 체크
- 

In [ ]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + '/fold_dataset/train_fold_1_of_5.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + '/fold_dataset/validation_fold_1_of_5.json' # train json 정보
cfg.data.val.pipeline[1]['img_scale'] = (512,512) # Resize


cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2) #schduler에 존재
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [ ]:
# build_dataset
datasets = [build_dataset(cfg.data.train), build_dataset(cfg.data.val)]

In [ ]:
# train dataset 확인
datasets[0]

In [ ]:
# valid dataset 확인
datasets[1]

In [ ]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

### 평가 Metric 관련
- bbox_mAP
- bbox_mAP_50 : iou = 0.5 기준
- bbox_mAP_75 : iou = 0.75 기준
- bbox_mAP_s : small objcet
- bbox_mAP_m : medium object
- bbox_mAP_l : large obeject

In [ ]:
# 모델 학습
#distributed -> 분산 학습을 위한 파라미터 True로 할경우 key_error 'LOCAL RANK' 발생(local rank가 없어서 발생하는 문제로 보임)
train_detector(model, datasets, cfg, distributed=False, validate=True)